# robomech2025 アニメーション

経路の作成

In [34]:
import numpy as np
import math

theta = 3*np.pi/4
l = 3.0

# section 1
x1 = np.linspace(0, 1, 100)
y1 = np.zeros_like(x1)

# section 2
x2 = np.linspace(1.0, 1.0+l*math.cos(theta), 100)
y2 = np.linspace(0.0, l*math.sin(theta), 100)

# section 3
x3 = np.linspace(1.0+l*math.cos(theta), 4.0+l*math.cos(theta), 100)
y3 = np.ones_like(x3) * l * math.sin(theta)

x = np.concatenate([x1, x2, x3])
y = np.concatenate([y1, y2, y3])
path = np.c_[x, y]

シミュレーション

In [35]:
import matplotlib
matplotlib.use('Agg')  # GUI を使わないバックエンドに設定
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyArrowPatch
import matplotlib.animation as animation

from collections import defaultdict
from config import DT, method_name_list, V_MAX, V_MIN, W_MAX, W_MIN
from pure_pursuit import pure_pursuit
from robot import forward_simulation_differential

plt.rcParams["font.size"] = 18  # 全体のフォントサイズが変更されます。
plt.rcParams["font.family"] = "Times New Roman"  # 全体のフォントを設定
plt.rcParams["mathtext.fontset"] = "stix"  # math fontの設定
plt.rcParams["font.weight"] = "normal"  # フォントの太さを細字に設定
plt.rcParams["axes.linewidth"] = 1.0  # axis line width
plt.rcParams["axes.grid"] = True  # make grid
plt.rcParams["legend.edgecolor"] = "black"  # edgeの色を変更
plt.rcParams["legend.handlelength"] = 1  # 凡例の線の長さを調節

In [36]:
def draw_animation(path: np.ndarray, robot_poses: np.ndarray, look_ahead_positions: np.ndarray, method_name:str, path_name: str):
    print("drawing animations...")

    fig, ax = plt.subplots(figsize=(3.5, 3.5))
    ax.set_xlim(np.min(path[:, 0]) - 0.5, np.max(path[:, 0]) + 0.5)
    ax.set_ylim(np.min(path[:, 1]) - 0.5, np.max(path[:, 1]) + 1)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    # ax.set_title(f'{method_name_dict[method_name]} Simulation')
    ax.set_aspect('equal')

    # プロット要素の初期化
    if method_name == "pp":
        path_color = "tab:blue"
    elif method_name == "app":
        path_color = "tab:orange"
    elif method_name == "rpp":
        path_color = "tab:green"
    elif method_name == "dwpp":
        path_color = "tab:red"
    path_line, = ax.plot(path[:, 0], path[:, 1], 'k--', label='Path', lw=2)  # パス
    robot_point, = ax.plot([], [], 'bo', label='Robot', lw=3)  # ロボット位置
    robot_path, = ax.plot([], [], color=path_color, lw=3)  # ロボット位置
    look_ahead_point, = ax.plot([], [], 'ro', label='Look Ahead', lw=3)  # ルックアヘッド位置

    robot_xs = []
    robot_ys = []

    # ロボットの向きを示す矢印
    robot_arrow = FancyArrowPatch((0, 0), (0, 0), mutation_scale=20, color='blue')
    ax.add_patch(robot_arrow)

    def init():
        robot_point.set_data([], [])
        robot_path.set_data([], [])
        look_ahead_point.set_data([], [])
        robot_arrow.set_visible(False)
        return robot_point, robot_path, look_ahead_point, robot_arrow

    def update(frame):
        # ロボットの位置を更新
        robot_x = robot_poses[frame, 0]
        robot_y = robot_poses[frame, 1]
        robot_theta = robot_poses[frame, 2]
        
        robot_xs.append(robot_x)
        robot_ys.append(robot_y)

        robot_point.set_data([robot_x], [robot_y])
        robot_path.set_data(robot_xs, robot_ys)

        # ロボットの向きを示す矢印を更新
        arrow_length = 0.5  # 矢印の長さ
        dx = arrow_length * np.cos(robot_theta)
        dy = arrow_length * np.sin(robot_theta)

        # 矢印の位置を更新
        robot_arrow.set_positions((robot_x, robot_y), (robot_x + dx, robot_y + dy))
        robot_arrow.set_visible(True)

        # ルックアヘッドの位置を更新
        if frame < len(robot_poses) - 1:
            look_ahead_x = look_ahead_positions[frame, 0]
            look_ahead_y = look_ahead_positions[frame, 1]
            look_ahead_point.set_data([look_ahead_x], [look_ahead_y])
        else:
            look_ahead_point.set_data([], [])

        return path_line, robot_point, look_ahead_point, robot_arrow

    ani = FuncAnimation(
        fig, update, frames=len(robot_poses), init_func=init, blit=False, interval=100, repeat=False
    )

    # 凡例を表示
    # ax.legend()

    # アニメーションを表示または保存
    # plt.show()
    ani.save(f'../results/{path_name}/{method_name}.mp4', writer='ffmpeg', fps=int(1/DT))
    plt.close()

In [37]:
import os
def draw_velocity_profile_animation(
    time_stamps: np.ndarray,
    robot_velocities: np.ndarray,
    robot_ref_velocities: np.ndarray,
    method_name: str,
    path_name: str,
):
    """
    Make two separate animated MP4s:
      • “…_v_profile.mp4”      (linear velocity v vs. time)
      • “…_omega_profile.mp4”  (angular velocity ω vs. time)

    Folder “…/results/<path_name>” is created if it does not exist.
    """

    # --- 1. unpack -----------------------------------------------------------
    t   = np.asarray(time_stamps).ravel()
    v   = robot_velocities[:, 0]
    w   = robot_velocities[:, 1]
    v_r = robot_ref_velocities[:, 0]
    w_r = robot_ref_velocities[:, 1]

    mean_dt     = float(np.mean(np.diff(t)))
    interval_ms = mean_dt * 1000.0
    fps         = max(1, int(round(1.0 / mean_dt)))

    out_dir = f"../results/{path_name}"
    os.makedirs(out_dir, exist_ok=True)

    # --- 2. helper function --------------------------------------------------
    def build_anim(y, y_ref, y_label, file_suffix, y_min_lim, y_max_lim):
        fig, ax = plt.subplots(figsize=(3, 1.5))

        # data lines (animated)
        line,     = ax.plot([], [], lw=3, label=y_label, c="red")
        line_ref, = ax.plot([], [], lw=3, ls="--", label=f"{y_label}_ref", c="blue")

        # fixed horizontal limit lines
        ax.axhline(y=y_max_lim, ls="--", c="black")
        ax.axhline(y=y_min_lim, ls="--", c="black")

        # axes cosmetics
        ax.set_xlim(-1.0, 21.0)
        ax.set_ylim(y_min_lim-0.1, y_max_lim+0.1)
        ax.set_xlabel("$t$")
        ax.set_ylabel(f"${y_label}$")
        ax.grid(True)

        # animation callbacks
        def init():
            line.set_data([], [])
            line_ref.set_data([], [])
            return line, line_ref

        def update(frame):
            idx = frame + 1
            line.set_data(t[:idx], y[:idx])
            line_ref.set_data(t[:idx], y_ref[:idx])
            return line, line_ref

        ani = animation.FuncAnimation(
            fig,
            update,
            frames=len(t),
            init_func=init,
            interval=interval_ms,
            blit=True,
            repeat=False,
        )

        ani.save(
            os.path.join(out_dir, f"{method_name}_{file_suffix}.mp4"),
            writer="ffmpeg",
            fps=fps,
        )
        plt.close(fig)

    # --- 3. build both animations -------------------------------------------
    build_anim(v, v_r, "v", "v_profile", V_MIN, V_MAX)
    build_anim(w, w_r, r"\omega", "omega_profile", W_MIN, W_MAX)

In [38]:
def simulation(path: np.ndarray, path_name: str, initial_pose: np.ndarray)-> tuple[dict, dict]:
    # 一つの曲線に対するシミュレーションと描画

    robot_poses_dict = defaultdict(list)
    robot_velocities_dict = defaultdict(list)
    robot_ref_velocities_dict = defaultdict(list)
    look_ahead_positions_dict = defaultdict(list)
    break_constraints_flags_dict = defaultdict(list)
    curvatures_flags_dict = defaultdict(list)
    regulated_vs_flags_dict = defaultdict(list)
    time_stamp_dict = defaultdict(list)

    for method_name in method_name_list:
        # print(f"Method: {method_name}")
        
        # 初期設定
        ## ロボットの状態
        current_pose = initial_pose
        current_velocity = np.array([0.0, 0.0])
        # データの保存
        robot_poses = [current_pose]
        robot_velocities = [current_velocity]
        robot_ref_velocities = [current_velocity]
        look_ahead_positions = []
        break_constraints_flags = [[False, False]]
        curvatures = [0.0]
        regulated_vs = [0.0]
        time_stamps = [0.0]
        
        while True:
            # 終了条件
            if current_velocity[0] == 0.0 and current_velocity[1] == 0.0 and len(robot_velocities) > 1:
                break
            
            next_velocity_ref, look_ahead_pos, break_constraints_flag,\
                curvature, regulated_v = pure_pursuit(current_pose, current_velocity, path, method_name)
            next_pose, next_velocity = forward_simulation_differential(current_pose, current_velocity, next_velocity_ref)
            
            robot_poses.append(next_pose)
            robot_velocities.append(next_velocity)
            robot_ref_velocities.append(next_velocity_ref)
            look_ahead_positions.append(look_ahead_pos)
            break_constraints_flags.append(break_constraints_flag)
            curvatures.append(curvature)
            regulated_vs.append(regulated_v)
            time_stamps.append(time_stamps[-1] + DT)
            
            current_pose = next_pose
            current_velocity = next_velocity
            
        # print(f"Simulation time: {sim_end_time - sim_start_time}s")
        
        robot_poses_dict[method_name] = robot_poses
        robot_velocities_dict[method_name] = robot_velocities
        robot_ref_velocities_dict[method_name] = robot_ref_velocities
        look_ahead_positions_dict[method_name] = look_ahead_positions
        break_constraints_flags_dict[method_name] = break_constraints_flags
        curvatures_flags_dict[method_name] = curvatures
        regulated_vs_flags_dict[method_name] = regulated_vs
        time_stamp_dict[method_name] = time_stamps
    
    for method_name in method_name_list:
        robot_poses = np.array(robot_poses_dict[method_name])
        look_ahead_positions = np.array(look_ahead_positions_dict[method_name])
        robot_velocities = np.array(robot_velocities_dict[method_name])
        robot_ref_velocities = np.array(robot_ref_velocities_dict[method_name])
        break_constraints_flags = np.array(break_constraints_flags_dict[method_name])
        curvatures = np.array(curvatures_flags_dict[method_name])
        regulated_vs = np.array(regulated_vs_flags_dict[method_name])
        time_stamps = np.array(time_stamp_dict[method_name])
        
        # 速度プロファイルの描画
        print(time_stamps[-1])
        draw_velocity_profile_animation(time_stamps, robot_velocities, robot_ref_velocities, method_name, path_name)
        # break
        # 各手法ごとのアニメーションの描画
        # draw_animation(path, robot_poses, look_ahead_positions, method_name, path_name)
        
    
    return

In [39]:
initial_pose = np.array([0.0, 0.0, 0.0])
simulation(path, "robomech2025", initial_pose)

18.00000000000012
17.100000000000108
20.65000000000016
19.900000000000148
